In [11]:
import pandas as pd
import numpy as np
import math as math
import matplotlib.pyplot as plt
import seaborn as sns
import random
%matplotlib inline 

In [12]:
# to see all the columns when using head() or describe()
pd.set_option('display.max_columns',60)

In [13]:
df = pd.read_csv("../diaries_transactions_all.csv", dtype={'account_startclose_balance': str})

In [14]:
dfb = df[(df["trx_family_code"]=="FRMLN")&((df["trx_type_code"]==2760)|(df["trx_type_code"]==3247))]

# Get rid of loans of with unexplained balance adjustments

In [15]:
dfb["trx_prx_purpose"].unique()

array(['6. Closing Balance--End of last DQ',
       '1. Starting balance (today)', '5. Interest accruing',
       '3. Payments', 'INCREASE--Unexplained balance adjustment',
       '2. New borrowinng (individual portion)', '4. Any known fees',
       'DECREASE--Unexplained balance adjustment',
       '7. Refund from lender', '2. New borrowing'], dtype=object)

We insert a new column that is 0 if there is no unexplained balance adjustment for this account and a 1 if there is one. The column in 1 or 0 for all entries that correspond to that account.

In [16]:
dfb.insert(dfb.shape[1], "acc_unexplained_trx", np.zeros(dfb.shape[0]))

This program goes through all unique accounts, checks whether it has an unexplained balance adjustment and if yes sets the variable u to 1. Then it goes through another for-loop and sets the entries of that column 0 for all accounts that have an unexplained balance adjustment.

In [17]:
for acc in dfb["account_ids"].unique():
    u = 0
    for index,row in dfb[dfb["account_ids"]==acc].iterrows():
        if row["trx_prx_purpose"]=="DECREASE--Unexplained balance adjustment" or row["trx_prx_purpose"]=="INCREASE--Unexplained balance adjustment":
            u = 1
    for index,row in dfb[dfb["account_ids"]==acc].iterrows():
        dfb.at[index,"acc_unexplained_trx"] = u

### Let us check if it worked.

In [18]:
s = set()
for acc in dfb[dfb["acc_unexplained_trx"]==0]["account_ids"].unique():
     for i in dfb[dfb["account_ids"]==acc]["trx_prx_purpose"].unique():
            s.add(i)
print(s)

{'1. Starting balance (today)', '7. Refund from lender', '3. Payments', '6. Closing Balance--End of last DQ', '4. Any known fees', '5. Interest accruing', '2. New borrowinng (individual portion)', '2. New borrowing'}


### Let's now only use the loans that have no unexplained balance adjustments.

In [19]:
dfb = dfb[dfb["acc_unexplained_trx"]==0]

# Truncation into chunks with no new borrowing

We first insert a column for the new account numbers. These will be like the old account numbers but with a .1 or .2 for each sequence after a new borrowing.

In [20]:
del dfb["new_account_ids"]

KeyError: 'new_account_ids'

In [21]:
dfb.insert(df.columns.get_loc("account_ids")+1,"new_account_ids", ["-"]*(dfb.shape[0]))

In [22]:
dfb.sample(1)

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,int_yr_mo,first_int_date,account_ids,new_account_ids,unique_accnts,m_ids_owner,unique_hm_owner,account_bsheet_desig,account_startclose_balance,account_formal,account_liquid,first_trx_date_acc,last_trx_date_acc,tot_acc_daysofobs,tot_acc_monthsofobs,trx_id,m_ids_trx,trx_date,trx_month,trx_year,trx_yr_mo,trx_dq_round,trx_stdtime_days_hh,trx_stdtime_mnths_hh,trx_stdtime_days_acc,trx_stdtime_mnths_acc,trx_class_code,trx_class_desc,trx_family_code,trx_family_desc,trx_type_code,trx_type_desc,trx_prx_purpose,trx_prx_purpose_fd,trx_fee,trx_bsheet_direction,trx_mode_code,trx_mode_desc,trx_place_incommunity,trx_distance_km,trx_outlet,trx_direction,trx_value_kes,trx_value_usd,ddd_gift,trx_inkind_units,trx_inkind_value_usd,trx_inkind_value_kes,trx_stdtime_mnths_hh_nr,acc_unexplained_trx
358262,KVIHC30,NaN,14aug2012,25sep2013,407,14,06=Cleaning interview,24sep2013,9,2013,2013_09,12sep2012,59134745004500000,-,NaN,59134381215900000,NaN,Liability,NaN,Formal,NaN,24oct2012,23sep2013,334,11,34138708685000000,59134381215900000,01jan2013,1,2013,2013_01,19.0,140,4,69,2,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,3247,Joint liability loan,3. Payments,Repayment on financial liability,0.0,Decrease,1.0,Cash,1.0,0.0,"07=Group meeting,07=Group meeting",Outflow,6000.0,70.58823,0,NaN,NaN,NaN,4.827586,0.0


In [23]:
acc = random.choice(dfb[dfb["trx_prx_purpose"]=="2. New borrowing"]["account_ids"].unique())

In [24]:
print(acc)

58134763288300000


acc = 59135900709000000

In [25]:
lna = dfb[dfb["account_ids"]==acc]

In [26]:
lna = lna.sort_values("trx_stdtime_days_acc")

In [27]:
lna

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,int_yr_mo,first_int_date,account_ids,new_account_ids,unique_accnts,m_ids_owner,unique_hm_owner,account_bsheet_desig,account_startclose_balance,account_formal,account_liquid,first_trx_date_acc,last_trx_date_acc,tot_acc_daysofobs,tot_acc_monthsofobs,trx_id,m_ids_trx,trx_date,trx_month,trx_year,trx_yr_mo,trx_dq_round,trx_stdtime_days_hh,trx_stdtime_mnths_hh,trx_stdtime_days_acc,trx_stdtime_mnths_acc,trx_class_code,trx_class_desc,trx_family_code,trx_family_desc,trx_type_code,trx_type_desc,trx_prx_purpose,trx_prx_purpose_fd,trx_fee,trx_bsheet_direction,trx_mode_code,trx_mode_desc,trx_place_incommunity,trx_distance_km,trx_outlet,trx_direction,trx_value_kes,trx_value_usd,ddd_gift,trx_inkind_units,trx_inkind_value_usd,trx_inkind_value_kes,trx_stdtime_mnths_hh_nr,acc_unexplained_trx
120752,KELDL13,NaN,01oct2012,25oct2013,389,13,04=Diaries Interview,15oct2012,10,2012,2012_10,28aug2012,58134763288300000,-,1.0,58134382346100000,NaN,Liability,NaN,Formal,NaN,03oct2012,25oct2013,387,13,58135227075000000,58134382346100000,03oct2012,10,2012,2012_10,1.0,2,0,0,0,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agriculture Loan,3. Payments,Repayment on financial liability,0.0,Decrease,1.0,Cash,1.0,0.0,07=Group meeting,Outflow,4500.0,52.94118,0,NaN,NaN,NaN,0.068965,0.0
320783,KELDL13,NaN,01oct2012,25oct2013,389,13,04=Diaries Interview,21jan2013,1,2013,2013_01,28aug2012,58134763288300000,-,NaN,58134382346100000,NaN,Liability,NaN,Formal,NaN,03oct2012,25oct2013,387,13,58136007875500000,58134382346100000,14dec2012,12,2012,2012_12,3.0,74,2,72,2,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agriculture Loan,3. Payments,Repayment on financial liability,0.0,Decrease,1.0,Cash,1.0,0.0,07=Group meeting,Outflow,1750.0,20.58824,0,NaN,NaN,NaN,2.551724,0.0
24434,KELDL13,NaN,01oct2012,25oct2013,389,13,06=Cleaning interview,03oct2013,10,2013,2013_10,28aug2012,58134763288300000,-,NaN,58134382346100000,NaN,Liability,NaN,Formal,NaN,03oct2012,25oct2013,387,13,34138312905800000,58134382346100000,29jan2013,1,2013,2013_01,17.0,120,4,118,4,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agriculture Loan,5. Interest accruing,NaN,0.0,Increase,16.0,Automatic,NaN,NaN,NaN,NaN,5000.0,58.82353,0,NaN,NaN,NaN,4.137931,0.0
121632,KELDL13,NaN,01oct2012,25oct2013,389,13,06=Cleaning interview,03oct2013,10,2013,2013_10,28aug2012,58134763288300000,-,NaN,58134382346100000,NaN,Liability,NaN,Formal,NaN,03oct2012,25oct2013,387,13,34138312903200000,58134382346100000,29jan2013,1,2013,2013_01,17.0,120,4,118,4,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agriculture Loan,2. New borrowing,New borrowing on financial liability,0.0,Increase,1.0,Cash,1.0,0.0,07=Group meeting,Inflow,20000.0,235.29410,0,NaN,NaN,NaN,4.137931,0.0
93404,KELDL13,NaN,01oct2012,25oct2013,389,13,04=Diaries Interview,15may2013,5,2013,2013_05,28aug2012,58134763288300000,-,NaN,58134382346100000,NaN,Liability,NaN,Formal,NaN,03oct2012,25oct2013,387,13,50136861279400000,58134382346100000,05mar2013,3,2013,2013_03,6.0,155,5,153,5,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agriculture Loan,3. Payments,Repayment on financial liability,0.0,Decrease,1.0,Cash,1.0,0.0,07=Group meeting,Outflow,2100.0,24.70588,0,NaN,NaN,NaN,5.344828,0.0
252028,KELDL13,NaN,01oct2012,25oct2013,389,13,04=Diaries Interview,15may2013,5,2013,2013_05,28aug2012,58134763288300000,-,NaN,58134382346100000,NaN,Liability,NaN,Formal,NaN,03oct2012,25oct2013,387,13,50136861282200000,58134382346100000,02apr2013,4,2013,2013_04,6.0,183,6,181,6,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agr

## Now that I see this account I realize that we actually need to consider that there might be transactions on the same day. So we need to check whether there is a new borrowing on any given day, not any given row.

In [28]:
a = lna["account_ids"].unique()[0]
print(type(a))
a = a+".1"
print(a)
print('{:.20f}'.format(float(a)))

<class 'str'>
58134763288300000.1
58134763288300000.00000000000000000000


For some reason the program does not account for the .1 in the float. Maybe it cannot handle that many digits? But I guess it is fine if we have the account numbers as strings.

In [29]:
dfb["trx_prx_purpose"].unique()

array(['6. Closing Balance--End of last DQ',
       '1. Starting balance (today)', '5. Interest accruing',
       '2. New borrowinng (individual portion)', '3. Payments',
       '4. Any known fees', '7. Refund from lender', '2. New borrowing'],
      dtype=object)

In [30]:
# we set the first account number to be the old account number.1
# the subsequent chunks will be called #.2, #.3, etc
i=1
acc = lna["account_ids"].unique()[0]
a = lna["account_ids"].unique()[0]+".1"
for index, row in lna.iterrows():
    if row["trx_prx_purpose"] == "2. New borrowinng (individual portion)" or row["trx_prx_purpose"] == "2. New borrowing":
        i +=1
        a = acc+"."+str(i)
        lna.at[index,"new_account_ids"]=a
    else:
         lna.at[index,"new_account_ids"]=a
        

In [31]:
lna[["account_ids","new_account_ids","trx_stdtime_days_acc","trx_prx_purpose","acc_unexplained_trx"]]

,account_ids,new_account_ids,trx_stdtime_days_acc,trx_prx_purpose,acc_unexplained_trx
120752,58134763288300000,58134763288300000.1,0,3. Payments,0.0
320783,58134763288300000,58134763288300000.1,72,3. Payments,0.0
24434,58134763288300000,58134763288300000.1,118,5. Interest accruing,0.0
121632,58134763288300000,58134763288300000.2,118,2. New borrowing,0.0
93404,58134763288300000,58134763288300000.2,153,3. Payments,0.0
252028,58134763288300000,58134763288300000.2,181,3. Payments,0.0
144046,58134763288300000,58134763288300000.2,216,3. Payments,0.0
348826,58134763288300000,58134763288300000.2,253,3. Payments,0.0
140888,58134763288300000,58134763288300000.2,299,3. Payments,0.0
5203,58134763288300000,58134763288300000.2,387,6. Closing Balance--End of last DQ,0.0


In [32]:
lna["new_account_ids"].unique()

array(['58134763288300000.1', '58134763288300000.2'], dtype=object)

### Now let's adjust this such that new borrowings are always the first thing on one day.

In [33]:
for acc in dfb["account_ids"]:
    i=0
    indices = []
    index_bor = 0
    lna = dfb[dfb["account_ids"]==acc]
    lna = lna.sort_values("trx_stdtime_days_acc")
    for d in lna["trx_stdtime_days_acc"].unique():
        if lna[lna["trx_stdtime_days_acc"]==d].shape[0]==1:
            indices.append(lna[lna["trx_stdtime_days_acc"]==d].index[0])
        elif lna[lna["trx_stdtime_days_acc"]==d].shape[0]>1:
            if lna[(lna["trx_stdtime_days_acc"]==d)&((lna["trx_prx_purpose"]=="2. New borrowing")|(lna["trx_prx_purpose"]=="2. New borrowing (individual portion)")|(lna["trx_prx_purpose"]=="1. Starting balance (today)"))].shape[0]>1:
                print("Check manually: several new borrowings on the same day.")
            elif lna[(lna["trx_stdtime_days_acc"]==d)&((lna["trx_prx_purpose"]=="2. New borrowing")|(lna["trx_prx_purpose"]=="2. New borrowing (individual portion)")|(lna["trx_prx_purpose"]=="1. Starting balance (today)"))].shape[0]==1:
                for kind in ["2. New borrowing","2. New borrowing (individual portion)","1. Starting balance (today)"]:
                    if kind in lna[lna["trx_stdtime_days_acc"]==d]["trx_prx_purpose"].unique():
                        index_bor = lna[(lna["trx_stdtime_days_acc"]==d) & (lna["trx_prx_purpose"]==kind)].index[0]
                        indices.append(index_bor)
                        for i in lna[lna["trx_stdtime_days_acc"]==d].index.drop(index_bor):
                           indices.append(i)
            else:
                for i in lna[lna["trx_stdtime_days_acc"]==d].index:
                    indices.append(i)
    # here we reindex only the lna dataset, it would be nice if we could reindex the whole dataset. 
    # But maybe it does not matter actually.
    lna = lna.loc[indices,]
    v=1
    acc = lna["account_ids"].unique()[0]
    a = lna["account_ids"].unique()[0]+".1"
    for index, row in lna.iterrows():
        if row["trx_prx_purpose"] == "2. New borrowinng (individual portion)" or row["trx_prx_purpose"] == "2. New borrowing":
            v +=1
            a = acc+"."+str(v)
            dfb.at[index,"new_account_ids"]=a
        else:
             dfb.at[index,"new_account_ids"]=a
    if acc == "59134942469700000":
        break

In [34]:
lna["trx_prx_purpose"]=="2. New borrowing"

23025     False
277437    False
149656    False
427271     True
19200     False
260728    False
288513    False
288508    False
248010    False
208412    False
105122    False
426300    False
242426    False
426114    False
314426    False
21202     False
Name: trx_prx_purpose, dtype: bool

In [35]:
dfb[dfb["account_ids"]=="59134942469700000"].sort_values("trx_stdtime_days_acc")

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,int_yr_mo,first_int_date,account_ids,new_account_ids,unique_accnts,m_ids_owner,unique_hm_owner,account_bsheet_desig,account_startclose_balance,account_formal,account_liquid,first_trx_date_acc,last_trx_date_acc,tot_acc_daysofobs,tot_acc_monthsofobs,trx_id,m_ids_trx,trx_date,trx_month,trx_year,trx_yr_mo,trx_dq_round,trx_stdtime_days_hh,trx_stdtime_mnths_hh,trx_stdtime_days_acc,trx_stdtime_mnths_acc,trx_class_code,trx_class_desc,trx_family_code,trx_family_desc,trx_type_code,trx_type_desc,trx_prx_purpose,trx_prx_purpose_fd,trx_fee,trx_bsheet_direction,trx_mode_code,trx_mode_desc,trx_place_incommunity,trx_distance_km,trx_outlet,trx_direction,trx_value_kes,trx_value_usd,ddd_gift,trx_inkind_units,trx_inkind_value_usd,trx_inkind_value_kes,trx_stdtime_mnths_hh_nr,acc_unexplained_trx
23025,KVIHC35,NaN,05aug2012,22sep2013,413,14,04=Diaries Interview,09oct2012,10,2012,2012_10,07aug2012,59134942469700000,59134942469700000.1,1.0,59134311227800000,NaN,Liability,Start,Formal,NaN,09oct2012,20sep2013,346,11,59134976561100000,59134311227800000,09oct2012,10,2012,2012_10,1.0,65,2,0,0,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agriculture Loan,1. Starting balance (today),NaN,0.0,Increase,18.0,STARTING BALANCE,NaN,NaN,NaN,NaN,108000.0,1270.58800,0,NaN,NaN,NaN,2.241379,0.0
277437,KVIHC35,NaN,05aug2012,22sep2013,413,14,04=Diaries Interview,05nov2012,11,2012,2012_11,07aug2012,59134942469700000,59134942469700000.1,NaN,59134311227800000,NaN,Liability,NaN,Formal,NaN,09oct2012,20sep2013,346,11,59135211421200000,59134311227800000,18oct2012,10,2012,2012_10,2.0,74,2,9,0,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agriculture Loan,3. Payments,Repayment on financial liability,0.0,Decrease,1.0,Cash,1.0,0.000000,07=Group meeting,Outflow,12000.0,141.17650,0,NaN,NaN,NaN,2.551724,0.0
149656,KVIHC35,NaN,05aug2012,22sep2013,413,14,04=Diaries Interview,03dec2012,12,2012,2012_12,07aug2012,59134942469700000,59134942469700000.1,NaN,59134311227800000,NaN,Liability,NaN,Formal,NaN,09oct2012,20sep2013,346,11,59135453246700000,59134311227800000,18nov2012,11,2012,2012_11,3.0,105,3,40,1,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agriculture Loan,3. Payments,Repayment on financial liability,0.0,Decrease,1.0,Cash,1.0,0.000000,07=Group meeting,Outflow,12000.0,141.17650,0,NaN,NaN,NaN,3.620690,0.0
19200,KVIHC35,NaN,05aug2012,22sep2013,413,14,06=Cleaning interview,21sep2013,9,2013,2013_09,07aug2012,59134942469700000,59134942469700000.2,NaN,59134311227800000,NaN,Liability,NaN,Formal,NaN,09oct2012,20sep2013,346,11,34138710898300000,59134311227800000,15dec2012,12,2012,2012_12,20.0,132,4,67,2,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agriculture Loan,5. Interest accruing,NaN,0.0,Increase,16.0,Automatic,NaN,NaN,NaN,NaN,2000.0,23.52941,0,NaN,NaN,NaN,4.551724,0.0
427271,KVIHC35,NaN,05aug2012,22sep2013,413,14,06=Cleaning interview,21sep2013,9,2013,2013_09,07aug2012,59134942469700000,59134942469700000.2,NaN,59134311227800000,NaN,Liability,NaN,Formal,NaN,09oct2012,20sep2013,346,11,34138710797700000,59134311227800000,15dec2012,12,2012,2012_12,20.0,132,4,67,2,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agriculture Loan,2. New borrowing,New borrowing on financial liability,0.0,Increase,3.0,"Transfer into bank, SACCO, or MFI account",0.0,2.189854,"06=Direct deposit, payroll deduction, or stand...",Inflow,10000.0,117.64710,0,NaN,NaN,NaN,4.551724,0.0
260728,KVIHC35,NaN,05aug2012,22sep2013,413,14,04=Diaries Interview,07jan2013,1,2013,2013_01,07aug2012,59134942469700000,59134942469700000.2,NaN,59134311227800000,NaN,Liability,NaN,Formal,NaN,09oct2012,20sep2013,34